# 06 - Dimensional Modeling: Symbol, Date, and Sentiment Dimensions

##  Notebook Purpose

This notebook creates core **dimension tables** to support a star schema design for the crypto sentiment vs price correlation project. These dimensions provide structured metadata for slicing, filtering, and enriching the main fact table: `gold_sentiment_price_combined`.

##  Dimension Tables Created

###  `dim_symbol`
A reference table that maps cryptocurrency tickers (`symbol`) to human-readable names and categories such as "Layer 1", "Meme", or "Exchange Token". Enables clear labeling in dashboards and grouping by coin type.

| symbol | coin_name     | category           |
|--------|---------------|--------------------|
| BTC    | Bitcoin       | Layer 1            |
| DOGE   | Dogecoin      | Meme               |
| BNB    | Binance Coin  | Exchange           |
| …      | …             | …                  |

---

### `dim_date`
A full date dimension covering every day in 2025. It includes common calendar attributes such as weekday name, month name, and a weekend flag. Useful for filtering, grouping, and time-series analysis.

| event_date | month_name | weekday_name | is_weekend |
|------------|------------|---------------|------------|
| 2025-01-01 | January    | Wednesday     | false      |
| 2025-01-04 | January    | Saturday      | true       |

---

###  `dim_sentiment_direction`
Maps technical sentiment direction values from your pipeline (`positive_day`, `neutral_day`, etc.) to readable labels and optional dashboard color codes.

| sentiment_direction | sentiment_label | color_code |
|---------------------|------------------|------------|
| positive_day        | Positive         | #2ecc71    |
| negative_day        | Negative         | #e74c3c    |
| neutral_day         | Neutral          | #95a5a6    |

---

##  Benefits

- Enables **semantic joins** in your dashboards or BI tools  
- Supports **star schema** design for better performance and clarity  
- Makes sentiment, symbol, and date dimensions **explorable and human-readable**

---

*Run this notebook once before querying the fact table or building dashboard joins.*


In [0]:
USE CATALOG tabular;
USE SCHEMA dataexpert;

In [0]:
-- ========================================================================================
-- dim_symbol: Lookup table for crypto symbols
-- Contains human-readable names and categories for each crypto asset
-- Supports semantic joins and grouping by coin type in dashboards
-- ========================================================================================
CREATE OR REPLACE TABLE sakethg_capstone_project_1_dim_symbol AS
SELECT * FROM VALUES
  ('BTC', 'Bitcoin', 'Layer 1'),
  ('ETH', 'Ethereum', 'Layer 1'),
  ('BNB', 'Binance Coin', 'Exchange'),
  ('SOL', 'Solana', 'Layer 1'),
  ('XRP', 'Ripple', 'Payments'),
  ('DOGE', 'Dogecoin', 'Meme'),
  ('ADA', 'Cardano', 'Layer 1'),
  ('DOT', 'Polkadot', 'Interoperability')
AS symbol_table(symbol, coin_name, category);


-- ========================================================================================
-- dim_date: Full calendar dimension for the year 2025
-- Includes useful calendar breakdowns (month, weekday, is_weekend)
-- Enables date filtering and grouping in fact tables and dashboards
-- ========================================================================================
CREATE OR REPLACE TABLE sakethg_capstone_project_1_dim_date AS
WITH calendar AS (
  -- Generate all dates for 2025
  SELECT explode(sequence(to_date('2025-01-01'), to_date('2025-12-31'), interval 1 day)) AS event_date
)
SELECT
  event_date,
  year(event_date) AS year,
  month(event_date) AS month,
  day(event_date) AS day,
  date_format(event_date, 'MMMM') AS month_name,
  CASE EXTRACT(DAYOFWEEK FROM event_date)
    WHEN 1 THEN 'Sunday'
    WHEN 2 THEN 'Monday'
    WHEN 3 THEN 'Tuesday'
    WHEN 4 THEN 'Wednesday'
    WHEN 5 THEN 'Thursday'
    WHEN 6 THEN 'Friday'
    WHEN 7 THEN 'Saturday'
  END AS weekday_name,
  CASE WHEN EXTRACT(DAYOFWEEK FROM event_date) IN (1, 7) THEN true ELSE false END AS is_weekend -- Mark weekends (1 = Sunday, 7 = Saturday)
FROM calendar;


-- ========================================================================================
-- dim_sentiment_direction: Maps internal sentiment codes to user-friendly labels and colors
-- Useful for consistent coloring and filtering in visualizations
-- ========================================================================================
CREATE OR REPLACE TABLE sakethg_capstone_project_1_dim_sentiment_direction AS
SELECT * FROM VALUES
  ('positive_day', 'Positive', '#2ecc71'),
  ('negative_day', 'Negative', '#e74c3c'),
  ('neutral_day',  'Neutral',  '#95a5a6')
AS t(sentiment_direction, sentiment_label, color_code);